In [1]:
import pandas as pd
import os
import re
import warnings
from tqdm.notebook import tqdm
import multiprocessing

#warnings.filterwarnings("ignore")

In [2]:
def process(base_dir,dir,file_name,index_val): 
    df_list = []
    with open(base_dir+dir+'/'+file_name,'r') as f:
        text = f.read()
        text = re.sub('[^A-Za-z0-9\?\.\!\;\,\s\r\t\n]',' ',text)
        sentences = re.split("\n\n",text)
        for sentence in sentences:
            sentence = re.sub('(\.[^0-9])|(\.[\s\n])',' . ',sentence)
            sentence = re.sub('\;',' ; ',sentence)
            sentence = re.sub(':',' : ',sentence)
            sentence = re.sub('\?',' ? ',sentence)
            sentence = re.sub('\!',' ! ',sentence)
            sentence = re.sub('\,',' , ',sentence)
            sentence = re.sub('[\n\r\t],',' ',sentence)
            sentence = sentence.strip()
            sentence = sentence +' [END_GEN]'
            while sentence.find('  ') > -1:
                sentence = sentence.replace('  ',' ')
            splits = sentence.split(' ')
            for index,s_split in enumerate(splits):
                if len(splits) != 2:
                    if not index == len(splits) - 1: 
                        if index >= 5:
                            temp_dict = {'Text':' '.join(splits[0:index+1]),'NW':splits[index+1]}
                            df_list.append(temp_dict)
                else:
                    if not index == len(splits) - 1:
                        temp_dict = {'Text':splits[0],'NW':splits[index+1]}
                        df_list.append(temp_dict)
    df = pd.DataFrame(df_list)
    df.to_csv(base_dir+'NWP_Books.csv', index = False, mode='a',header = False)
    with open(base_dir+'FilesCompleted.txt','w+') as f2:
        f2.write(str(index_val))

In [3]:
base_dir = 'D:/Data/'
dirs = os.listdir(base_dir)
for dir in dirs[0:1]:
    files = os.listdir(base_dir+dir)
    p = multiprocessing.Pool()
    for file_name in tqdm(files,total=len(files)):
        p.apply_async(process, [base_dir,dir,file_name,files.index(file_name)]) 
    p.close()
    p.join()

  0%|          | 0/196604 [00:00<?, ?it/s]